In [1]:
# # # #installing necessary libraries
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00


# **Loading dataset**

In [2]:

from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/final_train_40kdata_cat3.csv",split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
dataset

Dataset({
    features: ['productId', 'Title', 'userId', 'Helpfulness', 'Score', 'Time', 'input', 'Cat1', 'Cat2', 'output', 'instruction', 'text'],
    num_rows: 38983
})

In [4]:
from huggingface_hub import notebook_login

notebook_login()

# **Loading Model from Hugging Face**

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# **Loading Tokenizer**

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# **Importing necessary libraries from pef and setting up necessary parameters for fine tuning**

In [15]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# **Importing TrainingArguments from transformers and making necessary set up for fine tuning**

In [16]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)


# **importing SFTTrainer from trl and making necessry setup for fine tuning the model**

In [17]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/38983 [00:00<?, ? examples/s]

In [18]:

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# **Training starting**

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.778100
20,2.206100
30,1.722300
40,1.388400
50,0.992700
60,1.937400
70,1.672000
80,1.473400
90,1.243500
100,0.947500


TrainOutput(global_step=100, training_loss=1.6361436367034912, metrics={'train_runtime': 1053.8041, 'train_samples_per_second': 1.518, 'train_steps_per_second': 0.095, 'total_flos': 5863460936908800.0, 'train_loss': 1.6361436367034912, 'epoch': 0.04})

# **Saving the model locally in outputs directory**

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

# **Loading Fine tuned Lora model**

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

# **Pusing the model to hugging face hub**

In [ ]:
model.push_to_hub("llama2-fine-tuned-classfier-cat3")

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sakil/llama2-fine-tuned-classfier-cat3/commit/e2848503c3935ebd071965861d9b38a41d7d86f9', commit_message='Upload model', commit_description='', oid='e2848503c3935ebd071965861d9b38a41d7d86f9', pr_url=None, pr_revision=None, pr_num=None)

**Inferencing : Input to your prompt will be a text from column name Text, and output should be class name
from Column Name Cat 3**

In [20]:
# !pip install langchain

# **Loading fine-tuned model from hugging face**

In [21]:

lora_config = LoraConfig.from_pretrained('Sakil/llama2-fine-tuned-classfier-cat3')
model = get_peft_model(model, lora_config)

adapter_config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

# **Importing necessary libraries**

In [22]:
from huggingface_hub.hf_api import HfFolder
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

# **Calling text-generation from transformer pipeline**

In [23]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_length=60,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.2})

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

In [24]:
import pandas as pd

# **Finally performing inferencing on our inference data by fine tuned model and few shot prompting**

In [29]:

from langchain import PromptTemplate, LLMChain

# Read categories from the CSV file
df = pd.read_csv('/content/inference_data.csv')  # Replace 'your_csv_file.csv' with the actual filename
df['Text']=df['Text'].astype(str)
df=df.iloc[:100]
# Assuming the categories are in the 'cat2' column
categories = df['Cat3'].tolist()

# Create a comma-separated string of categories for the prompt
categories_str = ', '.join(categories)

# Modify the prompt template
template = f"""Classify the text into {categories_str}. Reply with only the words given in in: {categories_str}.

Examples:
Text: My eight year old loves this game, whenever he can coax it away from his father! I recommend you buy two so no one has to wait a whole afternoon for his or her turn. END.
Cat3: board games.

Text: This was a great book!!!! It is well thought through, and you can easily imagine the events happening. The Westing Game itself is a great way to tie two things together. The events are well sequenced and exciting. Ellen Raskin wrote a wonderful book
Cat3: unknown.

Text: My husband and I treat severly emotionally disturbed children in our home. Part of the treatment includes helping these children with their social skills. This game promotes family togetherness, as well as just plain fun. Even if you lose you still feel like youv'e won! p.s. what a great way to be able to shop. Most of the children we take care of have never been to the grocery store, much less a toy store. So usually when you take them it ends up being so overwhelming. What a blessing to be able to calmly shop at home, thank-you.Sincerely Rebecca Bidelspach
Cat3: card games.

Text: I'm Back! This time I'm going to try to explain what the game is. Ok lets get started.&quot; Its The year is 1923. You are one of four world famous rival archaeologists who have come to Cairo,Egypt to begin a race against time, nature, and each other, in search of the stolen treasures of the pharaoh. Enter the Vally of the Kings, a world of burning desert heat and blinding sandstorms. Danger lurks in your every step as you dig your way through the dark, damp corridors of ancient tombs. but beware.....the cobra goddess WADJET has gaurded the pharoahs' tombs for over 3000 years.....She awaits your intrusion.....! You will be faced with danger, intrigue, and possible financial ruin. You must use your wits,knowledge, money, and memory to outsmart your rivals and WADJET.....Search for royal tombs of the pharoahs.....learn their secrets.....But don't let anyone else learn yours!&quot;That about covers it doesn't it? I hope you enjoy the game and found my reveiws useful. Look for more of my reveiws in other catagories. look for my bold openion and smiley :o) then you'll know its me...... Enjoy...
Cat3: board games.

Text: On my way to a great vacation in the Pocono's with my son and his family. Niita took out the toy and started playing with. I was amazed at what it can do. So I just mailed her about Extreme Bop It
Cat3: unknown.

Text: {{text}}
Cat3:"""

# Assuming you have already defined the 'llm' model
# llm = ...

# Create the prompt template
prompt = PromptTemplate(template=template, input_variables=["text"])

# Create the LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [30]:
# Create a new column 'predicted_label' in the DataFrame
df['predicted_label_cat3'] = ""

# Iterate through each row and make predictions
for index, row in df.iterrows():
    text = row['Text']  # Assuming 'Text' is the column name in your CSV file
    raw_llm_answer = llm_chain.run(text)

    # Assuming raw_llm_answer is a string, not a dictionary
    predicted_label = raw_llm_answer.strip()

    df.at[index, 'predicted_label_cat3'] = predicted_label

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Input length of input_ids is 1405, but `max_length` is set to 60. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Input length of input_ids is 1431, but `max_length` is set to 60. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Input length of input_ids is 1485, but `max_le

In [31]:
df.head()

,productId,Title,userId,Helpfulness,Score,Time,Text,Cat1,Cat2,Cat3,predicted_label_cat3
0,B000E46LYG,Golden Valley Natural Buffalo Jerky,A3MQDNGHDJU4MK,0/0,3,-1,The description and photo on this product need...,grocery gourmet food,meat poultry,jerky,unknown
1,B000GRA6N8,Westing Game,unknown,0/0,5,860630400,This was a great book!!!! It is well thought t...,toys games,games,unknown,board
2,B000GRA6N8,Westing Game,unknown,0/0,5,883008000,"I am a first year teacher, teaching 5th grade....",toys games,games,unknown,unknown
3,B000GRA6N8,Westing Game,unknown,0/0,5,897696000,I got the book at my bookfair at school lookin...,toys games,games,unknown,card
4,B00000DMDQ,I SPY A is For Jigsaw Puzzle 63pc,unknown,4-Feb,5,911865600,Hi! I'm Martine Redman and I created this puzz...,toys games,puzzles,jigsaw puzzles,puzz


**saving the result**

In [32]:
df.to_csv(r"/content/inference_data_result_categorical3.csv",index=False)